<a href="https://colab.research.google.com/github/YeChen/RevealAPITest/blob/main/reveal_api_test_5_permissions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import os
from google.colab import userdata
from IPython.display import display, HTML

# Step 1: Retrieve USER and PASSWORD secrets from Colab
username = userdata.get("DEMO_USER")
password = userdata.get("DEMO_PASSWORD")

if not username or not password:
    raise ValueError("Secrets USER and PASSWORD must be set in Colab.")

def print_color(message, color):
    display(HTML(f'<p style="color: {color};">{message}</p>'))

# Step 2: Define the authentication function
def authenticate(username, password):
    """
    Authenticates with the Reveal API using username and password.
    Returns loginSessionId and authenticatedUserId if authentication is successful.
    """
    login_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/login"
    login_data = {
        "username": username,
        "password": password
    }

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Reveal-API-Tester/1.0"
    }

    response = requests.post(login_url, json=login_data, headers=headers)
    response.raise_for_status()

    login_response = response.json()
    login_session_id = login_response.get("loginSessionId")
    authenticated_user_id = login_response.get("userId")

    if not login_session_id or not authenticated_user_id:
        raise ValueError("Failed to retrieve loginSessionId and userId from login response.")

    print("Authentication successful!")
    return login_session_id, authenticated_user_id

# Step 3: Define the function to retrieve projects
def get_projects(login_session_id, authenticated_user_id):
    """
    Retrieves a list of projects for the authenticated user and returns the first project's projectId.
    """
    projects_url = "https://consulting.us-east-1.reveal11.cloud/rest/api/v2/projects"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    params = {"userId": authenticated_user_id}
    response = requests.get(projects_url, headers=headers, params=params)
    response.raise_for_status()

    projects_response = response.json()
    projects = projects_response.get("results", [])

    if not projects:
        print("No projects found.")
        return None

    # Extract the projectId of the first project
    first_project = projects[0]
    project_id = first_project.get("projectId")
    project_name = first_project.get("projectName", "Unnamed Project")
    print(f"First project found: {project_name} (projectId: {project_id})")
    return project_id

# Step 4: Define the function to get users for a project
def get_project_users(login_session_id, project_id):
    """
    Retrieves a list of users for the specified projectId and returns the first user's userId.
    """
    users_url = f"https://consulting.us-east-1.reveal11.cloud/rest/api/v2/{project_id}/users"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    response = requests.get(users_url, headers=headers)
    response.raise_for_status()

    users_response = response.json()
    users = users_response.get("results", [])

    if not users:
        print("No users found for the project.")
        return None

    # Extract the userId of the first user
    first_user = users[0]
    project_user_id = first_user.get("userId")
    username = first_user.get("userName", "Unknown")
    print(f"First user found: {username} (userId: {project_user_id})")
    return project_user_id

# Step 5: Define the function to get permissions for a user
def get_user_permissions(login_session_id, project_id, project_user_id):
    """
    Retrieves permissions for a specified projectUserId in a given projectId.
    """
    permissions_url = f"https://consulting.us-east-1.reveal11.cloud/rest/api/v2/{project_id}/users/{project_user_id}/permissions"

    headers = {
        "Authorization": f"Bearer {login_session_id}",
        "incontrolauthtoken": login_session_id,
        "User-Agent": "Reveal-API-Tester/1.0",
        "Accept": "application/json"
    }

    response = requests.get(permissions_url, headers=headers)
    response.raise_for_status()

    # Parse response, assuming it's a list
    permissions = response.json()

    if permissions:
        print("Permissions for the user:")
        for permission in permissions:
            permission_id = permission.get("id", "Unknown")
            permission_name = permission.get("name", "Unknown")
            permission_category = permission.get("category", "Unknown")
            print(f"- ID: {permission_id}, Name: {permission_name}, Category: {permission_category}")
    else:
        print("No permissions found for the user.")

# Main Execution
try:
    # Step 1: Authenticate to get session ID and authenticated user ID
    session_id, authenticated_user_id = authenticate(username, password)

    # Step 2: Retrieve the first project ID
    project_id = get_projects(session_id, authenticated_user_id)

    # Step 3: If project_id is found, retrieve the first project user's ID
    if project_id:
        project_user_id = get_project_users(session_id, project_id)

        # Step 4: If project_user_id is found, retrieve and print permissions
        if project_user_id:
            get_user_permissions(session_id, project_id, project_user_id)

except requests.exceptions.RequestException as e:
    print_color(f"API request failed: {e}", "red")
except ValueError as e:
    print_color(f"Error: {e}", "red")


Authentication successful!
First project found: DanProject2 (projectId: 1795)
First user found: dhonors (userId: 372)
Permissions for the user:
- ID: 1, Name: Review Documents, Category: Review
- ID: 2, Name: Update Documents, Category: Review
- ID: 3, Name: Modify View, Category: Settings
- ID: 4, Name: Bulk Update Documents, Category: Review
- ID: 5, Name: Redact, Category: Review
- ID: 6, Name: Annotate, Category: Review
- ID: 7, Name: Print Documents, Category: Review
- ID: 8, Name: Email Documents, Category: Review
- ID: 9, Name: Export Documents, Category: Review
- ID: 10, Name: View Reports , Category: Reports
- ID: 11, Name: Assign Documents, Category: Admin
- ID: 12, Name: Manage Permissions, Category: Admin
- ID: 13, Name: Manage Groups, Category: Admin
- ID: 14, Name: Manage Roles, Category: Admin
- ID: 15, Name: Manage Tags, Category: Admin
- ID: 16, Name: Search, Category: Search
- ID: 17, Name: Power Search, Category: Search
- ID: 18, Name: Save Searches, Category: Search